<a href="https://colab.research.google.com/github/lwp20/columbia-accenture-knowledge-graph-for-supply-chains/blob/main/Text_Splitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
# pip install transformers

In [ ]:
# pip install sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import json
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
import tiktoken

In [ ]:
file_path = '/content/drive/MyDrive/Capstone Project/Jsons2' ##change here to your own file location

def read_jsons(file_path):
    '''
    all_contents: a dict contains {title:content},
    content: a dict contains {"page_x_text":text}
    '''
    all_contents = {}
    for file in os.listdir(file_path):
        if file.endswith(".json"):
            f = open(file_path+"/"+file)
            passage = json.load(f)
            article = passage['Article']
            text_label = 'plum_contents'
            if text_label not in passage:
                text_label = 'ocr_contents'
            contents = passage[text_label]
            f.close()

            all_contents[article] = contents
    return all_contents

all_contents = read_jsons(file_path)

In [ ]:
contents = all_contents ["1909.08021v2.Strategic_Formation_and_Reliability_of_Supply_Chain_Networks.pdf"] #can change title here
contents["page_1_text"] #can change page here

'1909.08021v2 [cs.GT] 14 Jan 2020  arX1v  Strategic Formation and Reliability of Supply Chain Networks  (Working Paper, January 15, 2020)  Victor Amelkin  Warren Center for Network and Data Sciences, Department of Electrical and Systems Engineering University of Pennsylvania, Philadelphia, PA 19104, vctrQseas.upenn.edu, https://victoramelkin.com  Rakesh Vohra  Warren Center for Network and Data Sciences, Department of Electrical and Systems Engineering, Department of Economics University of Pennsylvania, Philadelphia, PA 19104, rvohraQ@seas.upenn.edu, https://sites.google.com/site /quaerereverum9 /  Supply chains are the backbone of the global economy. Disruptions to them can be costly. Centrally managed supply chains invest in ensuring their resilience. Decentralized supply chains, however, must rely upon the self-interest of their individual components to maintain the resilience of the entire chain.  We examine the incentives that independent self-interested agents have in forming a 

### Split by Characters

In [ ]:
def chunk_by_char(all_contents = all_contents):
    '''
    chunks_by_char: a dict contains {title:chunked_content},
    content: a dict contains {"page_x_text":chunked_sentences},
    chunk by characters
    '''
    chunks_by_char = {}
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 100,
        chunk_overlap  = 20,
        length_function = len,
        is_separator_regex = False,
    )
    for (article,contents) in all_contents.items():
        page_chunks = dict()
        for (key, text) in contents.items():
            chunks = text_splitter.create_documents([text])
            page_chunks[key] = []
            for chunk in chunks:
                page_chunks[key].append(chunk.page_content)
        chunks_by_char[article] = page_chunks
    return chunks_by_char

chunks_by_char = chunk_by_char()

In [ ]:
chunks_by_char['1909.08021v2.Strategic_Formation_and_Reliability_of_Supply_Chain_Networks.pdf']

{'page_1_text': ['1909.08021v2 [cs.GT] 14 Jan 2020  arX1v  Strategic Formation and Reliability of Supply Chain',
  'of Supply Chain Networks  (Working Paper, January 15, 2020)  Victor Amelkin  Warren Center for',
  'Warren Center for Network and Data Sciences, Department of Electrical and Systems Engineering',
  'Systems Engineering University of Pennsylvania, Philadelphia, PA 19104, vctrQseas.upenn.edu,',
  'https://victoramelkin.com  Rakesh Vohra  Warren Center for Network and Data Sciences, Department of',
  'Department of Electrical and Systems Engineering, Department of Economics University of',
  'University of Pennsylvania, Philadelphia, PA 19104, rvohraQ@seas.upenn.edu,',
  'https://sites.google.com/site /quaerereverum9 /  Supply chains are the backbone of the global',
  'of the global economy. Disruptions to them can be costly. Centrally managed supply chains invest in',
  'chains invest in ensuring their resilience. Decentralized supply chains, however, must rely upon',
  'mu

### Split by Tokens

1. using tokentextsplitter

In [ ]:
from langchain.text_splitter import TokenTextSplitter

def chunk_by_token(all_contents = all_contents):
    '''
    chunks_by_char: a dict contains {title:chunked_content},
    content: a dict contains {"page_x_text":chunked_sentences},
    chunk by default tokens
    '''
    chunks_by_token = {}
    token_splitter = TokenTextSplitter(
        chunk_size = 20,
        chunk_overlap  = 5
    )
    for (article,contents) in all_contents.items():
        page_chunks = dict()
        for (key, text) in contents.items():
            chunks = token_splitter.split_text(text)
            page_chunks[key] = chunks
        chunks_by_token[article] = page_chunks
    return chunks_by_char

chunks_by_token = chunk_by_token()



In [ ]:
chunks_by_token['1909.08021v2.Strategic_Formation_and_Reliability_of_Supply_Chain_Networks.pdf']

{'page_1_text': ['1909.08021v2 [cs.GT] 14 Jan 2020  arX1v  Strategic Formation and Reliability of Supply Chain',
  'of Supply Chain Networks  (Working Paper, January 15, 2020)  Victor Amelkin  Warren Center for',
  'Warren Center for Network and Data Sciences, Department of Electrical and Systems Engineering',
  'Systems Engineering University of Pennsylvania, Philadelphia, PA 19104, vctrQseas.upenn.edu,',
  'https://victoramelkin.com  Rakesh Vohra  Warren Center for Network and Data Sciences, Department of',
  'Department of Electrical and Systems Engineering, Department of Economics University of',
  'University of Pennsylvania, Philadelphia, PA 19104, rvohraQ@seas.upenn.edu,',
  'https://sites.google.com/site /quaerereverum9 /  Supply chains are the backbone of the global',
  'of the global economy. Disruptions to them can be costly. Centrally managed supply chains invest in',
  'chains invest in ensuring their resilience. Decentralized supply chains, however, must rely upon',
  'mu

以下方法不知道为什么不能很好chunk 有待修正 但可以接别各种token方法
有需要再改

2. use encoder in titoken

In [ ]:
# encoding = tiktoken.get_encoding("cl100k_base")
# encoding = tiktoken.encoding_for_model("gpt-4")

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=0,
    encoding_name="cl100k_base" ## encoding_name = encoder in tiktoken
    )
page_chunks = dict()
for key, text in contents.items():
    chunks = text_splitter.create_documents([text])
    page_chunks[key] = []
    for chunk in chunks:
        page_chunks[key].append(chunk.page_content)

In [ ]:
len(page_chunks['page_1_text'])

1

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)

page_chunks = dict()
for key, text in contents.items():
    chunks = text_splitter.split_text(text)
    page_chunks[key] = chunks

In [ ]:
len(page_chunks['page_10_text'])

1

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=5, chunk_overlap=1
)
text = "hello world, how about you? thanks, I am fine.  the machine learning class. So what I wanna do today is just spend a little time going over the logistics of the class, and then we'll start to talk a bit about machine learning"
texts = text_splitter.split_text(text)
# texts = text_splitter.split_text(contents['page_1_text'])

print(texts)

["hello world, how about you? thanks, I am fine.  the machine learning class. So what I wanna do today is just spend a little time going over the logistics of the class, and then we'll start to talk a bit about machine learning"]


In [ ]:
len(contents['page_1_text'])

2560

In [ ]:
# can get embedding here using langchain, (including hugging face, openai, ....)

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

text = "This is a test document."

query_result = embeddings.embed_query(text)

doc_result = embeddings.embed_documents([text])